In [191]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import copy

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor
from xgboost import XGBClassifier

import statsmodels.api as sm

In [192]:
np.random.seed(4)

In [193]:
def preprocess_transaction_flag(df):
    df['transaction_flag'] = df['transaction_count'].apply(lambda x: 1 if x > 0 else 0)
    return df

In [194]:
# 학습용 데이터 읽기
test_df = pd.read_csv("data/test/test_df.csv")
# test_df

In [195]:
# target 데이터 읽기
columns = ['date', 'transaction_count', 'transaction_amount', 'transaction_flag']
target_df = pd.read_csv("./data/target/wallet_ranking_target_unnamed_top_100_250110.csv")

### preprocessing transaction flag

target_df = preprocess_transaction_flag(target_df)

target_df.columns = columns

In [196]:
# 날짜에 맞춰 자르기
start = (test_df['date'].values)[0]
end = (test_df['date'].values)[-1]

def trim_date(df, start, end):
    return df[(df['date']>=start) & (df['date']<=end)]

target_df = trim_date(target_df, start, end)

In [197]:
# target 데이터 적용
test_df['transaction_count'] = target_df['transaction_count']
test_df['transaction_amount'] = target_df['transaction_amount']
test_df['transaction_flag'] = target_df['transaction_flag']

test_df['target'] = test_df['transaction_amount']

test_df = test_df.drop(['transaction_count', 'transaction_amount', 'transaction_flag'], axis=1)
test_df.set_index('date', inplace=True)

In [198]:
test_df = test_df.fillna(test_df.mean())

In [199]:
shifted_columns = test_df.columns.difference(['target'])
shifted_df = test_df[shifted_columns].shift(1)

shifted_columns = shifted_df.columns + '_shifted'
shifted_df.columns = shifted_columns

combined_df = pd.concat([shifted_df], axis=1)
combined_df

,abs_returns_shifted,avg-block-size_shifted,avg-confirmation-time_shifted,blocks-size_shifted,close_shifted,cost-per-transaction_shifted,cost-per-transaction-percent_shifted,daily_afternoon_returns_shifted,daily_afternoon_volatility_shifted,daily_avg_afternoon_returns_shifted,daily_avg_afternoon_volume_shifted,daily_avg_dawn_returns_shifted,daily_avg_dawn_volume_shifted,daily_avg_morning_returns_shifted,daily_avg_morning_volume_shifted,daily_avg_night_returns_shifted,daily_avg_night_volume_shifted,daily_avg_price_shifted,daily_avg_quote_asset_volume_shifted,daily_avg_spread_shifted,daily_avg_trades_shifted,daily_avg_volume_shifted,daily_dawn_returns_shifted,daily_dawn_volatility_shifted,daily_high_shifted,daily_low_shifted,daily_morning_returns_shifted,daily_morning_volatility_shifted,daily_night_returns_shifted,daily_night_volatility_shifted,daily_quote_asset_volume_shifted,daily_sum_taker_buy_base_asset_volume_shifted,daily_sum_taker_buy_quote_asset_volume_shifted,daily_sum_trades_shifted,daily_volatility_shifted,daily_vwap_shifted,difficulty_shifted,estimated-transaction-volume_shifted,estimated-transaction-volume-usd_shifted,fees-usd-per-transaction_shifted,hash-rate_shifted,high_shifted,high_low_shifted,low_shifted,market-price_shifted,median-confirmation-time_shifted,miners-revenue_shifted,n-payments_shifted,n-payments-per-block_shifted,n-transactions_shifted,n-transactions-excluding-popular_shifted,n-transactions-per-block_shifted,n-transactions-total_shifted,n-unique-addresses_shifted,open_shifted,output-volume_shifted,returns_shifted,trade-volume_shifted,transaction-fees_shifted,transaction-fees-usd_shifted,volume_shifted
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02,0.023098,1.222708,212.466831,322474.100444,4285.08,77.428762,1.280016,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,2.534347e+13,164027.544213,1.630505e+09,4.139801,1.837495e+08,4485.39,284.65,4200.74,19194.260675,9.630790,2.380613e+07,6.700324e+05,4607.731346,319896.709677,314504.774194,2184.527176,5.877181e+08,604537.233636,4261.48,1.836027e+06,0.001612,3.253403e+08,83.348399,1.445047e+06,795.150377
2017-01-03,0.041238,0.944878,212.466831,97293.875547,4108.37,7.350606,0.763861,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,3.176884e+11,275190.258573,2.799811e+08,0.394819,2.526780e+06,4371.52,432.75,3938.77,997.360000,9.816667,2.138666e+06,6.700324e+05,4607.731346,290951.000000,287576.000000,1829.880503,1.839618e+08,529709.000000,4285.08,1.573982e+06,-0.041238,6.822471e+06,112.884147,1.148730e+05,1199.888264
2017-01-04,0.007694,0.906574,212.466831,97444.110856,4139.98,7.415466,0.656971,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,3.176884e+11,333533.011560,3.417543e+08,0.398843,2.589950e+06,4184.69,334.69,3850.00,1013.420000,12.350000,2.245225e+06,6.700324e+05,4607.731346,301664.000000,297536.000000,1839.414634,1.842534e+08,515024.000000,4108.37,1.950525e+06,0.007694,8.324526e+06

In [200]:
combined_df = combined_df.dropna()
combined_df

,abs_returns_shifted,avg-block-size_shifted,avg-confirmation-time_shifted,blocks-size_shifted,close_shifted,cost-per-transaction_shifted,cost-per-transaction-percent_shifted,daily_afternoon_returns_shifted,daily_afternoon_volatility_shifted,daily_avg_afternoon_returns_shifted,daily_avg_afternoon_volume_shifted,daily_avg_dawn_returns_shifted,daily_avg_dawn_volume_shifted,daily_avg_morning_returns_shifted,daily_avg_morning_volume_shifted,daily_avg_night_returns_shifted,daily_avg_night_volume_shifted,daily_avg_price_shifted,daily_avg_quote_asset_volume_shifted,daily_avg_spread_shifted,daily_avg_trades_shifted,daily_avg_volume_shifted,daily_dawn_returns_shifted,daily_dawn_volatility_shifted,daily_high_shifted,daily_low_shifted,daily_morning_returns_shifted,daily_morning_volatility_shifted,daily_night_returns_shifted,daily_night_volatility_shifted,daily_quote_asset_volume_shifted,daily_sum_taker_buy_base_asset_volume_shifted,daily_sum_taker_buy_quote_asset_volume_shifted,daily_sum_trades_shifted,daily_volatility_shifted,daily_vwap_shifted,difficulty_shifted,estimated-transaction-volume_shifted,estimated-transaction-volume-usd_shifted,fees-usd-per-transaction_shifted,hash-rate_shifted,high_shifted,high_low_shifted,low_shifted,market-price_shifted,median-confirmation-time_shifted,miners-revenue_shifted,n-payments_shifted,n-payments-per-block_shifted,n-transactions_shifted,n-transactions-excluding-popular_shifted,n-transactions-per-block_shifted,n-transactions-total_shifted,n-unique-addresses_shifted,open_shifted,output-volume_shifted,returns_shifted,trade-volume_shifted,transaction-fees_shifted,transaction-fees-usd_shifted,volume_shifted
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,0.023098,1.222708,212.466831,322474.100444,4285.08,77.428762,1.280016,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,2.534347e+13,164027.544213,1.630505e+09,4.139801,1.837495e+08,4485.39,284.65,4200.74,19194.260675,9.630790,2.380613e+07,6.700324e+05,4607.731346,319896.709677,314504.774194,2184.527176,5.877181e+08,604537.233636,4261.48,1.836027e+06,0.001612,3.253403e+08,83.348399,1.445047e+06,795.150377
2017-01-03,0.041238,0.944878,212.466831,97293.875547,4108.37,7.350606,0.763861,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,3.176884e+11,275190.258573,2.799811e+08,0.394819,2.526780e+06,4371.52,432.75,3938.77,997.360000,9.816667,2.138666e+06,6.700324e+05,4607.731346,290951.000000,287576.000000,1829.880503,1.839618e+08,529709.000000,4285.08,1.573982e+06,-0.041238,6.822471e+06,112.884147,1.148730e+05,1199.888264
2017-01-04,0.007694,0.906574,212.466831,97444.110856,4139.98,7.415466,0.656971,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,3.176884e+11,333533.011560,3.417543e+08,0.398843,2.589950e+06,4184.69,334.69,3850.00,1013.420000,12.350000,2.245225e+06,6.700324e+05,4607.731346,301664.000000,297536.000000,1839.414634,1.842534e+08,515024.000000,4108.37,1.950525e+06,0.007694,8.324526e+06,117.508697,1.207600e+05,381.309763
2017-01-05,0.012969,0.986063,212.466831,97592.788943,4086.29,7.000531,0.537835,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.00019

In [201]:
combined_df['target'] = test_df['target']
combined_df

C:\Users\slaye\AppData\Local\Temp\ipykernel_48100\4003880687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['target'] = test_df['target']


,abs_returns_shifted,avg-block-size_shifted,avg-confirmation-time_shifted,blocks-size_shifted,close_shifted,cost-per-transaction_shifted,cost-per-transaction-percent_shifted,daily_afternoon_returns_shifted,daily_afternoon_volatility_shifted,daily_avg_afternoon_returns_shifted,daily_avg_afternoon_volume_shifted,daily_avg_dawn_returns_shifted,daily_avg_dawn_volume_shifted,daily_avg_morning_returns_shifted,daily_avg_morning_volume_shifted,daily_avg_night_returns_shifted,daily_avg_night_volume_shifted,daily_avg_price_shifted,daily_avg_quote_asset_volume_shifted,daily_avg_spread_shifted,daily_avg_trades_shifted,daily_avg_volume_shifted,daily_dawn_returns_shifted,daily_dawn_volatility_shifted,daily_high_shifted,daily_low_shifted,daily_morning_returns_shifted,daily_morning_volatility_shifted,daily_night_returns_shifted,daily_night_volatility_shifted,daily_quote_asset_volume_shifted,daily_sum_taker_buy_base_asset_volume_shifted,daily_sum_taker_buy_quote_asset_volume_shifted,daily_sum_trades_shifted,daily_volatility_shifted,daily_vwap_shifted,difficulty_shifted,estimated-transaction-volume_shifted,estimated-transaction-volume-usd_shifted,fees-usd-per-transaction_shifted,hash-rate_shifted,high_shifted,high_low_shifted,low_shifted,market-price_shifted,median-confirmation-time_shifted,miners-revenue_shifted,n-payments_shifted,n-payments-per-block_shifted,n-transactions_shifted,n-transactions-excluding-popular_shifted,n-transactions-per-block_shifted,n-transactions-total_shifted,n-unique-addresses_shifted,open_shifted,output-volume_shifted,returns_shifted,trade-volume_shifted,transaction-fees_shifted,transaction-fees-usd_shifted,volume_shifted,target
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,0.023098,1.222708,212.466831,322474.100444,4285.08,77.428762,1.280016,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,2.534347e+13,164027.544213,1.630505e+09,4.139801,1.837495e+08,4485.39,284.65,4200.74,19194.260675,9.630790,2.380613e+07,6.700324e+05,4607.731346,319896.709677,314504.774194,2184.527176,5.877181e+08,604537.233636,4261.48,1.836027e+06,0.001612,3.253403e+08,83.348399,1.445047e+06,795.150377,0.000000
2017-01-03,0.041238,0.944878,212.466831,97293.875547,4108.37,7.350606,0.763861,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,3.176884e+11,275190.258573,2.799811e+08,0.394819,2.526780e+06,4371.52,432.75,3938.77,997.360000,9.816667,2.138666e+06,6.700324e+05,4607.731346,290951.000000,287576.000000,1829.880503,1.839618e+08,529709.000000,4285.08,1.573982e+06,-0.041238,6.822471e+06,112.884147,1.148730e+05,1199.888264,0.000000
2017-01-04,0.007694,0.906574,212.466831,97444.110856,4139.98,7.415466,0.656971,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.736578,25017.884537,19469.012445,0.000192,17.258190,0.805244,-0.000873,0.000162,25632.904464,24348.192068,0.000205,0.000159,0.000865,0.000173,1.678334e+09,34421.057699,8.314316e+08,1.487585e+06,0.000179,25006.063354,3.176884e+11,333533.011560,3.417543e+08,0.398843,2.589950e+06,4184.69,334.69,3850.00,1013.420000,12.350000,2.245225e+06,6.700324e+05,4607.731346,301664.000000,297536.000000,1839.414634,1.842534e+08,515024.000000,4108.37,1.950525e+06,0.007694,8.324526e+06,117.508697,1.207600e+05,381.309763,0.000000
2017-01-05,0.012969,0.986063,212.466831,97592.788943,4086.29,7.000531,0.537835,0.000166,0.000185,6.625148e-09,1.044955,3.925940e-08,0.686782,-5.228610e-08,0.758826,9.726415e-08,0.73657

In [202]:
# 스케일링
scaler = MinMaxScaler()

scaled_df = pd.DataFrame(scaler.fit_transform(combined_df), columns=combined_df.columns)
scaled_df

,abs_returns_shifted,avg-block-size_shifted,avg-confirmation-time_shifted,blocks-size_shifted,close_shifted,cost-per-transaction_shifted,cost-per-transaction-percent_shifted,daily_afternoon_returns_shifted,daily_afternoon_volatility_shifted,daily_avg_afternoon_returns_shifted,daily_avg_afternoon_volume_shifted,daily_avg_dawn_returns_shifted,daily_avg_dawn_volume_shifted,daily_avg_morning_returns_shifted,daily_avg_morning_volume_shifted,daily_avg_night_returns_shifted,daily_avg_night_volume_shifted,daily_avg_price_shifted,daily_avg_quote_asset_volume_shifted,daily_avg_spread_shifted,daily_avg_trades_shifted,daily_avg_volume_shifted,daily_dawn_returns_shifted,daily_dawn_volatility_shifted,daily_high_shifted,daily_low_shifted,daily_morning_returns_shifted,daily_morning_volatility_shifted,daily_night_returns_shifted,daily_night_volatility_shifted,daily_quote_asset_volume_shifted,daily_sum_taker_buy_base_asset_volume_shifted,daily_sum_taker_buy_quote_asset_volume_shifted,daily_sum_trades_shifted,daily_volatility_shifted,daily_vwap_shifted,difficulty_shifted,estimated-transaction-volume_shifted,estimated-transaction-volume-usd_shifted,fees-usd-per-transaction_shifted,hash-rate_shifted,high_shifted,high_low_shifted,low_shifted,market-price_shifted,median-confirmation-time_shifted,miners-revenue_shifted,n-payments_shifted,n-payments-per-block_shifted,n-transactions_shifted,n-transactions-excluding-popular_shifted,n-transactions-per-block_shifted,n-transactions-total_shifted,n-unique-addresses_shifted,open_shifted,output-volume_shifted,returns_shifted,trade-volume_shifted,transaction-fees_shifted,transaction-fees-usd_shifted,volume_shifted,target
0,0.058470,0.377989,0.008012,0.444296,0.015684,0.243694,0.206983,0.457019,0.079778,0.446828,0.074536,0.370856,0.082084,0.391316,0.088289,0.387934,0.061741,0.313856,0.096229,0.031992,0.097865,0.090975,0.518291,0.055147,0.317110,0.314049,0.628123,0.053607,0.652915,0.075963,0.096171,0.091907,0.094720,0.097794,0.078870,0.313557,0.270977,0.224842,0.102451,0.031372,0.245205,0.017147,0.018747,0.020196,0.275726,0.267287,0.209253,0.299529,0.344747,0.243350,0.238235,0.288313,0.446874,0.393149,0.015346,0.061314,0.639711,0.065635,0.052364,0.017250,0.000746,0.000000e+00
1,0.104388,0.245267,0.008012,0.000000,0.013155,0.005898,0.110809,0.457019,0.079778,0.446828,0.074536,0.370856,0.082084,0.391316,0.088289,0.387934,0.061741,0.313856,0.096229,0.031992,0.097865,0.090975,0.518291,0.055147,0.317110,0.314049,0.628123,0.053607,0.652915,0.075963,0.096171,0.091907,0.094720,0.097794,0.078870,0.313557,0.000000,0.408713,0.011325,0.001854,0.000512,0.015532,0.029674,0.016372,0.003241,0.274336,0.005161,0.299529,0.344747,0.207275,0.204733,0.212595,0.000000,0.296188,0.015684,0.050474,0.570604,0.001376,0.072178,0.000829,0.001279,0.000000e+00
2,0.019476,0.226969,0.008012,0.000296,0.013608,0.006118,0.090892,0.457019,0.079778,0.446828,0.074536,0.370856,0.082084,0.391316,0.088289,0.387934,0.061741,0.313856,0.096229,0.031992,0.097865,0.090975,0.518291,0.055147,0.317110,0.314049,0.628123,0.053607,0.652915,0.075963,0.096171,0.091907,0.094720,0.097794,0.078870,0.313557,0.000000,0.505216,0.015493,0.001886,0.000597,0.012882,0.022439,0.015077,0.003482,0.370417,0.006165,0.299529,0.344747,0.220626,0.217124,0.214631,0.000323,0.277159,0.013155,0.066050,0.649519,0.001679,0.075280,0.000902,0.000202,0.000000e+00
3,0.032828,0.264942,0.008012,0.000590,0.012840,0.004710,0.068694,0.457019,0.079778,0.446828,0.074536,0.370856,0.082084,0.391316,0.088289,0.387934,0.061741,0.313856,0.096229,0.031992,0.097865,0.090975,0.518291,0.055147,0.317110,0.314049,0.628123,0.053607,0.652915,0.075963,0.096171,0.091907,0.094720,0.097794,0.078870,0.313557,0.000000,0.610179,0.021695,0.003826,0.000384,0.013256,0.010913,0.017742,0.003847,0.307838,0.006987,0.299529,0.344747,0.254249,0.251531,0.280512,0.000658,0.316791,0.013608,0.088036,0.616196,0.001884,0.126150,0.002064,0.000314,0.000000e+00
4,0.043543,0.264050,0.008012,0.000887,0.011834,0.004414,0.034669,0.457019,0.07

In [203]:
y = scaled_df['target'].values
X = scaled_df.drop('target', axis=1)

In [204]:
## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [205]:
# shift 1, Linear Regression
lr = LinearRegression(fit_intercept=True)
lr.fit(X_train, y_train)
print('훈련 세트 R2 :', lr.score(X_train, y_train))

# 테스트 셋 예측
lr_y_pred = lr.predict(X_test)

# 정확도
predict_df = pd.DataFrame({'y_test': y_test, 'y_pred': lr_y_pred})
predict_df['diff'] = predict_df['y_test'] - predict_df['y_pred']
predict_df['diff_abs'] = predict_df['diff'].abs()
predict_df['diff_squared'] = predict_df['diff']**2
print("MAE:", predict_df['diff_abs'].mean())
print("MSE:", predict_df['diff_squared'].mean())
print("R2:", lr.score(X_test, y_test))

훈련 세트 R2 : 0.05374880215098554
MAE: 0.008967531385088873
MSE: 0.0013868347053673624
R2: -0.059518949571087276


In [206]:
# # 현재 모델
# XGBR = XGBRegressor(n_estimators=100000, gamma=0.006, subsample=0.70, learning_rate=0.08, max_depth=3, objective='reg:squarederror')
# XGBR.fit(X_train, y_train)

# predict_df = pd.DataFrame({'y_test': y_test, 'y_pred': XGBR.predict(X_test)})
# predict_df['diff'] = predict_df['y_test'] - predict_df['y_pred']
# predict_df['diff_abs'] = predict_df['diff'].abs()
# predict_df['diff_squared'] = predict_df['diff']**2
# print("MAE:", predict_df['diff_abs'].mean())
# print("MSE:", predict_df['diff_squared'].mean())
# print("R2:", XGBR.score(X_test, y_test))

In [207]:
# 현재 모델
XGBR = XGBRegressor(n_estimators=100000, gamma=0.002, subsample=0.90, learning_rate=0.08, max_depth=3, objective='reg:squarederror')
XGBR.fit(X_train, y_train)

predict_df = pd.DataFrame({'y_test': y_test, 'y_pred': XGBR.predict(X_test)})
predict_df['diff'] = predict_df['y_test'] - predict_df['y_pred']
predict_df['diff_abs'] = predict_df['diff'].abs()
predict_df['diff_squared'] = predict_df['diff']**2
print("MAE:", predict_df['diff_abs'].mean())
print("MSE:", predict_df['diff_squared'].mean())
print("R2:", XGBR.score(X_test, y_test))

MAE: 0.004864640024768022
MSE: 0.0013292305955109878
R2: -0.015510355230468731


In [208]:
XGBR.save_model('models/xgb_model_named_wallet_amount_250125.json')

In [209]:
# 모델 불러오기
XGBR = XGBRegressor()
XGBR.load_model('models/xgb_model_named_wallet_amount_250125.json')

# Error, R2 평가
predict_df = pd.DataFrame({'y_test': y_test, 'y_pred': XGBR.predict(X_test)})
predict_df['diff'] = predict_df['y_test'] - predict_df['y_pred']
predict_df['diff_abs'] = predict_df['diff'].abs()
predict_df['diff_squared'] = predict_df['diff']**2
print("MAE:", predict_df['diff_abs'].mean())
print("MSE:", predict_df['diff_squared'].mean())
print("R2:", XGBR.score(X_test, y_test))

MAE: 0.004864640024768022
MSE: 0.0013292305955109878
R2: -0.015510355230468731


In [210]:
xxx_amount = XGBR.predict(X)
test_X = copy.deepcopy(X)
test_X['ITI_amount'] = xxx_amount
test_X.index = combined_df.index
test_X

,abs_returns_shifted,avg-block-size_shifted,avg-confirmation-time_shifted,blocks-size_shifted,close_shifted,cost-per-transaction_shifted,cost-per-transaction-percent_shifted,daily_afternoon_returns_shifted,daily_afternoon_volatility_shifted,daily_avg_afternoon_returns_shifted,daily_avg_afternoon_volume_shifted,daily_avg_dawn_returns_shifted,daily_avg_dawn_volume_shifted,daily_avg_morning_returns_shifted,daily_avg_morning_volume_shifted,daily_avg_night_returns_shifted,daily_avg_night_volume_shifted,daily_avg_price_shifted,daily_avg_quote_asset_volume_shifted,daily_avg_spread_shifted,daily_avg_trades_shifted,daily_avg_volume_shifted,daily_dawn_returns_shifted,daily_dawn_volatility_shifted,daily_high_shifted,daily_low_shifted,daily_morning_returns_shifted,daily_morning_volatility_shifted,daily_night_returns_shifted,daily_night_volatility_shifted,daily_quote_asset_volume_shifted,daily_sum_taker_buy_base_asset_volume_shifted,daily_sum_taker_buy_quote_asset_volume_shifted,daily_sum_trades_shifted,daily_volatility_shifted,daily_vwap_shifted,difficulty_shifted,estimated-transaction-volume_shifted,estimated-transaction-volume-usd_shifted,fees-usd-per-transaction_shifted,hash-rate_shifted,high_shifted,high_low_shifted,low_shifted,market-price_shifted,median-confirmation-time_shifted,miners-revenue_shifted,n-payments_shifted,n-payments-per-block_shifted,n-transactions_shifted,n-transactions-excluding-popular_shifted,n-transactions-per-block_shifted,n-transactions-total_shifted,n-unique-addresses_shifted,open_shifted,output-volume_shifted,returns_shifted,trade-volume_shifted,transaction-fees_shifted,transaction-fees-usd_shifted,volume_shifted,ITI_amount
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,0.058470,0.377989,0.008012,0.444296,0.015684,0.243694,0.206983,0.457019,0.079778,0.446828,0.074536,0.370856,0.082084,0.391316,0.088289,0.387934,0.061741,0.313856,0.096229,0.031992,0.097865,0.090975,0.518291,0.055147,0.317110,0.314049,0.628123,0.053607,0.652915,0.075963,0.096171,0.091907,0.094720,0.097794,0.078870,0.313557,0.270977,0.224842,0.102451,0.031372,0.245205,0.017147,0.018747,0.020196,0.275726,0.267287,0.209253,0.299529,0.344747,0.243350,0.238235,0.288313,0.446874,0.393149,0.015346,0.061314,0.639711,0.065635,0.052364,0.017250,0.000746,0.000944
2017-01-03,0.104388,0.245267,0.008012,0.000000,0.013155,0.005898,0.110809,0.457019,0.079778,0.446828,0.074536,0.370856,0.082084,0.391316,0.088289,0.387934,0.061741,0.313856,0.096229,0.031992,0.097865,0.090975,0.518291,0.055147,0.317110,0.314049,0.628123,0.053607,0.652915,0.075963,0.096171,0.091907,0.094720,0.097794,0.078870,0.313557,0.000000,0.408713,0.011325,0.001854,0.000512,0.015532,0.029674,0.016372,0.003241,0.274336,0.005161,0.299529,0.344747,0.207275,0.204733,0.212595,0.000000,0.296188,0.015684,0.050474,0.570604,0.001376,0.072178,0.000829,0.001279,0.000944
2017-01-04,0.019476,0.226969,0.008012,0.000296,0.013608,0.006118,0.090892,0.457019,0.079778,0.446828,0.074536,0.370856,0.082084,0.391316,0.088289,0.387934,0.061741,0.313856,0.096229,0.031992,0.097865,0.090975,0.518291,0.055147,0.317110,0.314049,0.628123,0.053607,0.652915,0.075963,0.096171,0.091907,0.094720,0.097794,0.078870,0.313557,0.000000,0.505216,0.015493,0.001886,0.000597,0.012882,0.022439,0.015077,0.003482,0.370417,0.006165,0.299529,0.344747,0.220626,0.217124,0.214631,0.000323,0.277159,0.013155,0.066050,0.649519,0.001679,0.075280,0.000902,0.000202,0.000944
2017-01-05,0.032828,0.264942,0.008012,0.000590,0.012840,0.004710,0.068694,0.457019,0.079778,0.446828,0.074536,0.370856,0.082084,0.391316,0.088289,0.387934,0.061741,0.313856,0.096229,0.031992,0.097865,0.090975,0.518291,0.055147,0.317110,0.314049,0.628123,0.053607,0.652915,0.075963,0.096171,0.091907,0.094720,0.097794,0.078870,0.313557,0.000000,0.610179,0.021695,0.003826,0.000384,0.013256,0.010913,0.017742,0.003847,0.307838,0.006987,0.299529,0.344747,0.254249,0.251531,0.280512,0.000658,0.316791,0.013608,0.088036,0.616196,0.001884,0.126150,0.002064,0.000

In [211]:
test_X[['ITI_amount']].to_csv("data/iti/ITI_amount_250125.csv", index=True)

In [212]:
ITI_amount = pd.read_csv('data/iti/ITI_amount_250125.csv')
ITI_amount = ITI_amount.set_index('date')
ITI_amount

,ITI_amount
date,
2017-01-02,0.000944
2017-01-03,0.000944
2017-01-04,0.000944
2017-01-05,0.000944
2017-01-06,0.000944
...,...
2024-09-26,0.000944
2024-09-27,0.000944
2024-09-28,0.000944


### 학습시킨 transaction amount 예측

In [213]:
# # 변동성 확대
# reversion_df = pd.read_csv('reversion.csv')
# reversion_df = reversion_df[['Open datetime', 'Reversion_Signals']]
# reversion_df.columns = ['date', 'target']
# reversion_df = reversion_df.set_index('date')
# reversion_df

# 실제 트레이딩 확인 (변수명은 그대로)
reversion_df = target_df.copy()
reversion_df = reversion_df[['date', 'transaction_amount']]
reversion_df.columns = ['date', 'target']
reversion_df = reversion_df.set_index('date')

# 내일 target 예측
reversion_df = reversion_df.shift(-1).dropna()
reversion_df

,target
date,
2017-01-01,0.000000
2017-01-02,0.000000
2017-01-03,0.000000
2017-01-04,0.000000
2017-01-05,0.000000
...,...
2024-09-25,0.000055
2024-09-26,14600.002105
2024-09-27,0.000000


In [214]:
result_df = pd.concat([ITI_amount, reversion_df], axis=1)
result_df = result_df.dropna()
# result_df

# test_df

In [215]:
y = result_df['target'].shift(-1).dropna().values
X = result_df[['ITI_amount']][:-1]

In [216]:
## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=0)

In [217]:
X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
# print("=== OLS 기본 요약 ===")
# print(lr.summary())

# 로버스트 공분산(이분산 robust: HC1) 적용
# ---------------------------------------------
lr_hc1 = lr.get_robustcov_results(cov_type='HC1')
print("\n=== 로버스트(HC1) 요약 ===")
print(lr_hc1.summary())



=== 로버스트(HC1) 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                     2.281
Date:                Tue, 04 Feb 2025   Prob (F-statistic):              0.131
Time:                        18:42:34   Log-Likelihood:                -11323.
No. Observations:                 849   AIC:                         2.265e+04
Df Residuals:                     847   BIC:                         2.266e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.274e+04   6307.0

### Returns 예측

In [218]:
test_df = pd.read_csv("data/test/test_df.csv")
test_df = test_df.drop(['transaction_count', 'transaction_amount', 'transaction_flag'], axis=1)
test_df.set_index('date', inplace=True)

In [219]:
returns_df = (test_df.shift(-1))['returns'].dropna().abs()
# returns_df = test_df['returns'].shift(-1).dropna().abs()
# returns_df = test_df[['date', 'returns']].shift(-1).dropna()

result_df = pd.concat([ITI_amount, returns_df], axis=1)
result_df
result_df = result_df.dropna()

result_df.columns = ['ITI_amount', 'target']
result_df

,ITI_amount,target
date,,
2017-01-02,0.000944,0.007694
2017-01-03,0.000944,0.012969
2017-01-04,0.000944,0.017201
2017-01-05,0.000944,0.005976
2017-01-06,0.000944,0.018319
...,...,...
2024-09-25,0.000944,0.032018
2024-09-26,0.000944,0.009144
2024-09-27,0.000944,0.001339


In [220]:
y = result_df['target'].values
X = result_df[['ITI_amount']]

In [221]:
## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=0)

In [222]:
X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
# print("=== OLS 기본 요약 ===")
# print(lr.summary())

# 로버스트 공분산(이분산 robust: HC1) 적용
# ---------------------------------------------
lr_hc1 = lr.get_robustcov_results(cov_type='HC1')
print("\n=== 로버스트(HC1) 요약 ===")
print(lr_hc1.summary())



=== 로버스트(HC1) 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.7191
Date:                Tue, 04 Feb 2025   Prob (F-statistic):              0.397
Time:                        18:42:34   Log-Likelihood:                 2105.6
No. Observations:                 849   AIC:                            -4207.
Df Residuals:                     847   BIC:                            -4198.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0181      0.0

### 변동성 예측

In [223]:
test_df = pd.read_csv("data/test/test_df.csv")
test_df = test_df.drop(['transaction_count', 'transaction_amount', 'transaction_flag'], axis=1)
test_df.set_index('date', inplace=True)

test_df = test_df.fillna(test_df.mean())

test_target_df = pd.concat([ITI_amount, test_df], axis=1)

test_target_df = test_target_df.dropna()

test_target_df['volatility_5'] = test_target_df['returns'].rolling(window=5).std()
test_target_df['volatility_10'] = test_target_df['returns'].rolling(window=10).std()
test_target_df['volatility_15'] = test_target_df['returns'].rolling(window=15).std()
test_target_df['volatility_30'] = test_target_df['returns'].rolling(window=30).std()
test_target_df['volatility_60'] = test_target_df['returns'].rolling(window=60).std()

pd.set_option('display.max_columns', None)

# nan값 많은 열
# 'cost-per-transaction-percent'
# 'estimated-transaction-volume-usd'
# 'estimated-transaction-volume'

test_target_df = test_target_df.drop(['cost-per-transaction-percent', 'estimated-transaction-volume-usd', 'estimated-transaction-volume'], axis=1)

test_target_df

,ITI_amount,returns,close,abs_returns,volume,open,high,low,high_low,daily_avg_trades,daily_sum_trades,daily_sum_taker_buy_base_asset_volume,daily_sum_taker_buy_quote_asset_volume,daily_avg_spread,daily_quote_asset_volume,daily_avg_quote_asset_volume,daily_low,daily_high,daily_avg_price,daily_vwap,daily_avg_volume,daily_morning_returns,daily_afternoon_returns,daily_night_returns,daily_dawn_returns,daily_avg_morning_returns,daily_avg_afternoon_returns,daily_avg_night_returns,daily_avg_dawn_returns,daily_avg_morning_volume,daily_avg_afternoon_volume,daily_avg_night_volume,daily_avg_dawn_volume,daily_morning_volatility,daily_afternoon_volatility,daily_night_volatility,daily_dawn_volatility,daily_volatility,avg-block-size,blocks-size,cost-per-transaction,difficulty,fees-usd-per-transaction,hash-rate,market-price,median-confirmation-time,miners-revenue,n-transactions-excluding-popular,n-transactions-per-block,n-transactions-total,n-transactions,n-unique-addresses,output-volume,trade-volume,transaction-fees-usd,transaction-fees,n-payments-per-block,n-payments,avg-confirmation-time,volatility_5,volatility_10,volatility_15,volatility_30,volatility_60
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,0.000944,-0.041238,4108.37,0.041238,1199.888264,4285.08,4371.52,3938.77,432.75,17.258190,1.487585e+06,34421.057699,8.314316e+08,0.000192,1.678334e+09,19469.012445,24348.192068,25632.904464,25017.884537,25006.063354,0.805244,0.000205,0.000166,0.000865,-0.000873,-5.228610e-08,6.625148e-09,9.726415e-08,3.925940e-08,0.758826,1.044955,0.736578,0.686782,0.000159,0.000185,0.000173,0.000162,0.000179,0.944878,97293.875547,7.350606,3.176884e+11,0.394819,2.526780e+06,997.360000,9.816667,2.138666e+06,287576.0,1829.880503,1.839618e+08,290951.0,529709.0,1.573982e+06,6.822471e+06,114872.988609,112.884147,4607.731346,6.700324e+05,212.466831,NaN,NaN,NaN,NaN,NaN
2017-01-03,0.000944,0.007694,4139.98,0.007694,381.309763,4108.37,4184.69,3850.00,334.69,17.258190,1.487585e+06,34421.057699,8.314316e+08,0.000192,1.678334e+09,19469.012445,24348.192068,25632.904464,25017.884537,25006.063354,0.805244,0.000205,0.000166,0.000865,-0.000873,-5.228610e-08,6.625148e-09,9.726415e-08,3.925940e-08,0.758826,1.044955,0.736578,0.686782,0.000159,0.000185,0.000173,0.000162,0.000179,0.906574,97444.110856,7.415466,3.176884e+11,0.398843,2.589950e+06,1013.420000,12.350000,2.245225e+06,297536.0,1839.414634,1.842534e+08,301664.0,515024.0,1.950525e+06,8.324526e+06,120759.971352,117.508697,4607.731346,6.700324e+05,212.466831,NaN,NaN,NaN,NaN,NaN
2017-01-04,0.000944,-0.012969,4086.29,0.012969,467.083022,4139.98,4211.08,4032.62,178.46,17.258190,1.487585e+06,34421.057699,8.314316e+08,0.000192,1.678334e+09,19469.012445,24348.192068,25632.904464,25017.884537,25006.063354,0.805244,0.000205,0.000166,0.000865,-0.000873,-5.228610e-08,6.625148e-09,9.726415e-08,3.925940e-08,0.758826,1.044955,0.736578,0.686782,0.000159,0.000185,0.000173,0.000162,0.000179,0.986063,97592.788943,7.000531,3.176884e+11,0.644994,2.432026e+06,1037.830000,10.700000,2.332472e+06,325192.0,2147.986928,1.845561e+08,328642.0,545609.0,2.482038e+06,9.340584e+06,214902.266038,193.337436,4607.731346,6.700324e+05,212.466831,NaN,NaN,NaN,NaN,NaN
2017-01-05,0.000944,-0.017201,4016.00,0.017201,691.743060,4086.29,4119.62,3911.79,207.83,17.258190,1.487585e+06,34421.057699,8.314316e+08,0.000192,1.678334e+09,19469.012445,24348.192068,25632.904464,25017.884537,25006.063354,0.805244,0.000205,0.000166,0.000865,-0.000873,-5.228610e-08,6.625148e-09,9.726415e-08,3.925940e-08,0.758826,1.044955,0.736578,0.686782,0.000159,0.000185,0.000173,0.000162,0.000179,0.984197,97743.656720,6.913398,3.176884e+11,0.419516,2.210933e+06,1109.160000,10.916667,2.009801e+06,283920.0,2060.721429,1.848845e+08,288501.0,495910.0,2.998215e+06,2.387011e+07,121958.044759,114.991943,4607.731346,6.700324e+05,212.466831,NaN,NaN,NaN,NaN,NaN
2017-01-06,0.000944,0.005976,4040.00,0.005976,966.684858,4016.00,4104.82,3400.00,704.82,17.258190,1.487585e+06,34

In [224]:
y = test_target_df['volatility_5'].values
X = test_target_df['ITI_amount']

## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS 기본 요약 ===")
print(lr.summary())

=== OLS 기본 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     5.573
Date:                Tue, 04 Feb 2025   Prob (F-statistic):             0.0185
Time:                        18:42:34   Log-Likelihood:                 2423.2
No. Observations:                 849   AIC:                            -4842.
Df Residuals:                     847   BIC:                            -4833.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0226      0.001  

In [225]:
test_target_df.columns

Index(['ITI_amount', 'returns', 'close', 'abs_returns', 'volume', 'open',
       'high', 'low', 'high_low', 'daily_avg_trades', 'daily_sum_trades',
       'daily_sum_taker_buy_base_asset_volume',
       'daily_sum_taker_buy_quote_asset_volume', 'daily_avg_spread',
       'daily_quote_asset_volume', 'daily_avg_quote_asset_volume', 'daily_low',
       'daily_high', 'daily_avg_price', 'daily_vwap', 'daily_avg_volume',
       'daily_morning_returns', 'daily_afternoon_returns',
       'daily_night_returns', 'daily_dawn_returns',
       'daily_avg_morning_returns', 'daily_avg_afternoon_returns',
       'daily_avg_night_returns', 'daily_avg_dawn_returns',
       'daily_avg_morning_volume', 'daily_avg_afternoon_volume',
       'daily_avg_night_volume', 'daily_avg_dawn_volume',
       'daily_morning_volatility', 'daily_afternoon_volatility',
       'daily_night_volatility', 'daily_dawn_volatility', 'daily_volatility',
       'avg-block-size', 'blocks-size', 'cost-per-transaction', 'difficulty',

In [226]:
y = (test_target_df['returns'].shift(-1)).fillna(value=0).values
X = test_target_df[['ITI_amount', 'daily_avg_volume', 'high_low', 'volatility_5', 'daily_sum_taker_buy_quote_asset_volume']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS 기본 요약 ===")
print(lr.summary())

=== OLS 기본 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.3681
Date:                Tue, 04 Feb 2025   Prob (F-statistic):              0.871
Time:                        18:42:34   Log-Likelihood:                 1851.6
No. Observations:                 849   AIC:                            -3691.
Df Residuals:                     843   BIC:                            -3663.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------

In [227]:
y = (test_target_df['returns'].shift(-1)).fillna(value=0).values
X = test_target_df[['ITI_amount', 'high_low', 'volatility_5', 'volume']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS 기본 요약 ===")
print(lr.summary())

=== OLS 기본 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.1806
Date:                Tue, 04 Feb 2025   Prob (F-statistic):              0.948
Time:                        18:42:34   Log-Likelihood:                 1851.0
No. Observations:                 849   AIC:                            -3692.
Df Residuals:                     844   BIC:                            -3668.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0008      0

In [228]:
# # 자동으로 돌리기리기

# import time
# from IPython.display import clear_output
# for t in test_target_df.columns:
#     # 출력 지우기 (덮어쓰기 효과)
#     clear_output(wait=True)
#     y = test_target_df['returns'].values
#     y = (test_target_df['returns'].shift(-1)).fillna(value=0).values
#     X = test_target_df[['ITI_amount', t]]

#     ## train, test 데이터 분리
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

#     X_with_const = sm.add_constant(X_test)

#     # OLS 회귀 적합
#     # ---------------------------------------------
#     lr = sm.OLS(y_test, X_with_const).fit()
#     print("=== OLS 기본 요약 ===")
#     print(lr.summary())

#     # # 로버스트 공분산(이분산 robust: HC1) 적용
#     # # ---------------------------------------------
#     # lr_hc1 = lr.get_robustcov_results(cov_type='HC1')
#     # print("\n=== 로버스트(HC1) 요약 ===")
#     # print(lr_hc1.summary())


#     time.sleep(3)


# target 데이터 적용
test_df['transaction_count'] = target_threshold_3000_receive_df['transaction_count']
test_df['transaction_amount'] = target_threshold_3000_receive_df['transaction_amount']
test_df['transaction_flag'] = target_threshold_3000_receive_df['transaction_flag']

test_df['transaction_count'] = target_receive_df['transaction_count']
test_df['transaction_amount'] = target_receive_df['transaction_amount']
test_df['transaction_flag'] = target_receive_df['transaction_flag']

test_df['target'] = test_df['transaction_amount']

test_df = test_df.drop(['transaction_count', 'transaction_amount', 'transaction_flag'], axis=1)
test_df.set_index('date', inplace=True)

### Event 예측

In [229]:
# 비트코인 이벤트
event_df = pd.read_csv("data/event/bitcoin_event.csv")

# 학습용 데이터 읽기
test_df = pd.read_csv("data/test/test_df.csv")


# target 데이터 읽기
columns = ['date', 'transaction_count', 'transaction_amount', 'transaction_flag']
target_df = pd.read_csv("./data/target/wallet_ranking_target_unnamed_top_100_250110.csv")

### preprocessing transaction flag

target_df = preprocess_transaction_flag(target_df)

target_df.columns = columns
target_df = target_df.set_index('date').copy()

In [230]:
# 실제값 이벤트 예측용
test_target_df['transaction_amount'] = target_df['transaction_amount']
test_target_df['transaction_count'] = target_df['transaction_count']
test_target_df['transaction_flag'] = target_df['transaction_flag']

test_target_df['transaction_amount'] = test_target_df['transaction_amount'].apply(lambda x: 0.000001 if x == 0 else x)
test_target_df['transaction_amount_returns'] = test_target_df['transaction_amount'].pct_change(1).fillna(0)

In [233]:
y = (event_df[(event_df['Date']>=list(test_target_df.index)[0])&(event_df['Date']<=list(test_target_df.index)[-1])]['Event'])
y = y.shift(-1).fillna(0).values
# y = y.values
X = test_target_df[['transaction_amount']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS 기본 요약 ===")
print(lr.summary())

=== OLS 기본 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                   0.04341
Date:                Tue, 04 Feb 2025   Prob (F-statistic):              0.835
Time:                        18:44:56   Log-Likelihood:                 730.00
No. Observations:                 849   AIC:                            -1456.
Df Residuals:                     847   BIC:                            -1447.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const             